# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [1]:
!head data/smsspamcollection.csv

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [2]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./data/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,-1
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [3]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [4]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [5]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 148728.98it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [6]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

In [7]:
df_train.iloc[1,:]

text     Ok lar... Joking wif u oni...
label                               -1
Name: 1, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [8]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.1%
Coverage for free(): 4.6%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [9]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.0,0.0
free,1,[1],0.045536,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [10]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
3266,"44 7732584351, Do you want a New Nokia 3510i colour phone DeliveredTomorrow? With 300 free minutes to any mobile + 100 free texts + Free Camcorder reply or call 08000930705.",-1
495,Are you free now?can i call now?,-1
3595,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 08000930705,-1
1507,Thanks for the Vote. Now sing along with the stars with Karaoke on your mobile. For a FREE link just reply with SING now.,-1
4903,* FREE* POLYPHONIC RINGTONE Text SUPER to 87131 to get your FREE POLY TONE of the week now! 16 SN PoBox202 NR31 7ZS subscription 450pw,-1
2980,"Xmas Offer! Latest Motorola, SonyEricsson & Nokia & FREE Bluetooth or DVD! Double Mins & 1000 Txt on Orange. Call MobileUpd8 on 08000839402 or call2optout/4QF2",-1
3436,Hi darlin i cantdo anythingtomorrow as myparents aretaking me outfor a meal. when are u free? Katexxx,-1
2109,"FREE2DAY sexy St George's Day pic of Jordan!Txt PIC to 89080 dont miss out, then every wk a saucy celeb!4 more pics c PocketBabe.co.uk 0870241182716 £3/wk",-1
2705,FreeMsg: Fancy a flirt? Reply DATE now & join the UKs fastest growing mobile dating service. Msgs rcvd just 25p to optout txt stop to 83021. Reply DATE now!,-1
4298,thesmszone.com lets you send free anonymous and masked messages..im sending this message from there..do you see the potential for abuse???,-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [11]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 54310.89it/s]


In [12]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.000000,0.0
free,1,[1],0.045536,0.001346,0.0
call_now,2,[1],0.004486,0.001346,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [13]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets

{(-1, -1): array([   0,    1,    2, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   9,   71,   77,  113,  141,  143,  150,  176,  203,  224,  244,
         272,  298,  301,  326,  355,  385,  409,  445,  460,  472,  499,
         521,  534,  580,  612,  616,  628,  629,  637,  680,  744,  786,
         797,  842,  897,  917,  960,  964, 1005, 1071, 1119, 1136, 1146,
        1171, 1181, 1182, 1193, 1207, 1215, 1238, 1263, 1277, 1291, 1307,
        1328, 1377, 1384, 1389, 1392, 1400, 1449, 1492, 1508, 1523, 1549,
        1584, 1608, 1633, 1659, 1669, 1690, 1704, 1727, 1757, 1792, 1810,
        1820, 1829, 1830, 1865, 1873, 1881, 1933, 2000, 2010, 2053, 2061,
        2098, 2111, 2127, 2148, 2162, 2165, 2235, 2271, 2278, 2353, 2375,
        2380, 2389, 2396, 2401, 2491, 2494, 2505, 2531, 2607, 2644, 2647,
        2658, 2717, 2725, 2728, 2729, 2738, 2743, 2794, 2812, 2867, 2922,
        2937, 2950, 2964, 2978, 3005, 3009, 3017, 3019, 3055, 3084, 3085,
        3115, 3130, 3136, 3155, 3161, 31

In [14]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
876,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
1366,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
1502,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
2170,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
2871,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call.",-1
2992,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
4073,Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821,-1
4108,"HOT LIVE FANTASIES call now 08707500020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
4283,U can call now...,-1


In [15]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.000000,0.0
free,1,[1],0.045536,0.001346,0.0
call_now,2,[1],0.004486,0.001346,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [16]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [17]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"I\s.*free", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 67612.15it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.000000,0.000000
free,1,[1],0.045536,0.045536,0.005384
call_now,2,[1],0.004486,0.001346,0.000000
regex_I_am_free,3,"[0, 1]",0.045536,0.045536,0.005384


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [18]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

,text,label
3692,"Sir, i am waiting for your call, once free please call me.",-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
2121,"Argh my 3g is spotty, anyway the only thing I remember from the research we did was that province and sterling were the only problem-free places we looked at",-1
3778,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1
4949,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",-1
4151,I only work from mon to thurs but Sat i cant leh... Booked liao... Which other day u free?,-1
3436,Hi darlin i cantdo anythingtomorrow as myparents aretaking me outfor a meal. when are u free? Katexxx,-1
2225,"I prefer my free days... Tues, wed, fri oso can... Ü ask those workin lor...",-1
1663,Hi if ur lookin 4 saucy daytime fun wiv busty married woman Am free all next week Chat now 2 sort time 09099726429 JANINExx Calls£1/minMobsmoreLKPOBOX177HP51FL,-1
4587,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [19]:
@labeling_function()
def contains_money(sms):
    return SPAM if re.search(r"[$£€][0-9]", sms.text) else ABSTAIN

### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [20]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, contains_money]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 37386.47it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.001795,0.000000
free,1,[1],0.045536,0.045536,0.005384
call_now,2,[1],0.004486,0.004038,0.000000
regex_I_am_free,3,"[0, 1]",0.045536,0.045536,0.005384
has_many_uppercase_words,4,[1],0.180574,0.041050,0.001122
contains_money,5,[1],0.048901,0.030283,0.000673


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [21]:
@labeling_function()
def short_and_no_uppercase(sms):
    if sms.text.count(' ') < 9:
        for word in sms.text.split(' '):
            return HAM if word == word.isupper() else ABSTAIN
    return ABSTAIN

### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [22]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [23]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation: /Users/dawid/Desktop/PUT/Data
Mining/Data_Mining/.venv/lib/python3.12/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.7.2,<3.8.0   3.7.1   ✔



In [24]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [25]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents:
    print(e.text, e.label_)

England GPE
the United Kingdom GPE
Wales ORG
Scotland GPE
The Irish Sea LOC
England GPE
the Celtic Sea LOC
England GPE
Europe LOC
the North Sea LOC
English LANGUAGE
five-eighths CARDINAL
Great Britain GPE
the North Atlantic LOC
over 100 CARDINAL
the Isles of Scilly GPE
the Isle of Wight GPE


In [26]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [27]:
df_train.columns

Index(['text', 'label'], dtype='object')

In [28]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [29]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:22<00:00, 197.00it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.003140,0.001346
free,1,[1],0.045536,0.045536,0.006281
call_now,2,[1],0.004486,0.004038,0.000000
regex_I_am_free,3,"[0, 1]",0.045536,0.045536,0.006281
has_many_uppercase_words,4,[1],0.180574,0.026694,0.007851
has_person,5,[0],0.047555,0.008973,0.008973


Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [30]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [31]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [32]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 6523.85it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.003589,0.001346
free,1,[1],0.045536,0.045536,0.006281
call_now,2,[1],0.004486,0.004038,0.000000
regex_I_am_free,3,"[0, 1]",0.045536,0.045536,0.006281
has_many_uppercase_words,4,[1],0.180574,0.044639,0.007851
has_person,5,[0],0.047555,0.016375,0.016375
many_rare_words,6,[1],0.061238,0.032526,0.008748


In [33]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

,text,label
2775,"Dude u knw also telugu..thts gud..k, gud nyt..",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
4808,PRIVATE! Your 2004 Account Statement for 078498****7 shows 786 unredeemed Bonus Points. To claim call 08719180219 Identifier Code: 45239 Expires 06.05.05,-1
3388,So u workin overtime nigpun?,-1
4237,"U can WIN £100 of Music Gift Vouchers every week starting NOW Txt the word DRAW to 87066 TsCs www.Idew.com SkillGame, 1Winaweek, age16. 150ppermessSubscription",-1
713,08714712388 between 10am-7pm Cost 10p,-1
3582,Ard 4 lor...,-1
4572,"CHA QUITEAMUZING THATSCOOL BABE,PROBPOP IN & CU SATTHEN HUNNY 4BREKKIE! LOVE JEN XXX. PSXTRA LRG PORTIONS 4 ME PLEASE",-1
717,+449071512431 URGENT! This is the 2nd attempt to contact U!U have WON £1250 CALL 09071512433 b4 050703 T&CsBCM4235WC1N3XX. callcost 150ppm mobilesvary. max£7. 50,-1
2908,ALRITE,-1


#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [34]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

Yetunde    PROPN      NNP        Yetunde   
,          PUNCT      ,          ,         
i          PRON       PRP        I         
'm         AUX        VBP        be        
sorry      ADJ        JJ         sorry     
but        CCONJ      CC         but       
moji       ADJ        JJ         moji      
and        CCONJ      CC         and       
i          PRON       PRP        I         
seem       VERB       VBP        seem      
too        ADV        RB         too       
busy       ADJ        JJ         busy      
to         PART       TO         to        
be         AUX        VB         be        
able       ADJ        JJ         able      
to         PART       TO         to        
go         VERB       VB         go        
shopping   NOUN       NN         shopping  
.          PUNCT      .          .         


In [35]:
NLP_MODEL = spacy.load('en_core_web_sm')
@labeling_function()
def is_adjective(sms):
    return SPAM if sum(1 for token in NLP_MODEL(sms) if token.pos_ == "ADJ") > 3 else ABSTAIN

## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [36]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 1114/1114 [00:06<00:00, 180.01it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011216,0.001795,0.001346
free,1,[1],0.045536,0.045536,0.006281
call_now,2,[1],0.004486,0.001795,0.000000
regex_I_am_free,3,"[0, 1]",0.045536,0.045536,0.006281
has_person,4,[0],0.047555,0.010319,0.010319
many_rare_words,5,[1],0.061238,0.014581,0.008748


In [37]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.008977,0.000000,0.000000
free,1,[1],0.055655,0.055655,0.008977
call_now,2,[1],0.001795,0.000000,0.000000
regex_I_am_free,3,"[0, 1]",0.055655,0.055655,0.008977
has_person,4,[0],0.057451,0.009874,0.009874
many_rare_words,5,[1],0.071813,0.017056,0.009874


In [38]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [39]:
preds_train

array([-1,  1, -1, ..., -1, -1, -1])

In [40]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 3844
LABEL: 0, count: 166
LABEL: 1, count: 448


In [41]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.009]
INFO:root:[100 epochs]: TRAIN:[loss=0.003]
INFO:root:[200 epochs]: TRAIN:[loss=0.003]
INFO:root:[300 epochs]: TRAIN:[loss=0.002]
100%|██████████| 500/500 [00:00<00:00, 3877.56epoch/s]
INFO:root:Finished Training


In [42]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority voting accuracy: 50.4%
Probabilistic model accuracy: 50.9%


Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [43]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

((4458, 2), (677, 2))

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [44]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [45]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0)

In [46]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

Logistic regression accuracy: 53.3%


As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

In [47]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words, hot, contains_money, short_and_no_uppercase, ]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train, progress_bar=False)
L_test = applier.apply(df=df_test, progress_bar=False)
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100000, seed=42, progress_bar=False)
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"\n{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))
X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())
sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

INFO:root:Computing O...
INFO:root:Estimating \mu...
INFO:root:[0 epochs]: TRAIN:[loss=0.011]
INFO:root:Finished Training



Majority voting accuracy: 51.6%
Probabilistic model accuracy: 52.2%
Logistic regression accuracy: 46.9%


## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [48]:
import spacy

nlp = spacy.load('en_core_web_sm')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

Entities: []
Entities: []
Entities: [('Sunshine Quiz Wkly', 'PERSON'), ('Sony', 'ORG'), ('Algarve', 'ORG'), ('82277', 'DATE'), ('1.50', 'MONEY')]
Entities: [('07090298926', 'CARDINAL')]
Entities: [('zyada kisi ko kuch nahi milta', 'PERSON'), ('Zindgi', 'PERSON'), ('Zindgi wo hai', 'GPE'), ('jo ham jeetey hai', 'PERSON')]
Entities: [('2003', 'DATE'), ('07753741225', 'DATE'), ('800', 'CARDINAL'), ('un', 'ORG'), ('S. I. M.', 'PERSON'), ('42478', 'CARDINAL'), ('24/10/04', 'DATE')]
Entities: []
Entities: []
Entities: []
Entities: [('ALRITE SAM', 'ORG'), ('NIC', 'ORG')]
Entities: [('Yar', 'PERSON'), ('ge', 'ORG')]
Entities: [('2', 'CARDINAL'), ('One', 'CARDINAL')]
Entities: []
Entities: [('Babe', 'PERSON')]
Entities: []
Entities: [('a great month', 'DATE')]
Entities: [('4', 'CARDINAL')]
Entities: []
Entities: []
Entities: [('18 days', 'DATE'), ('daily', 'DATE')]
Entities: [('Yavnt', 'NORP')]
Entities: [('1', 'CARDINAL'), ('7', 'CARDINAL'), ('next saturday', 'DATE'), ('Maretare', 'ORG'), ('1'

In [49]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

['jurong point',
 'Nah',
 'Melle Melle',
 'Mark',
 'Yummy',
 'Mark',
 'Mallika Sherawat',
 'Divorce Barbie',
 'Ken',
 'Wah']

In [50]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacy])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [51]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [52]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [53]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [54]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [55]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/dawid/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/share/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [56]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

  0%|          | 0/446 [00:00<?, ?it/s]


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/dawid/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/share/nltk_data'
    - '/Users/dawid/Desktop/PUT/Data Mining/Data_Mining/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [57]:
df_train_sample.shape, df_train_augmented.shape

NameError: name 'df_train_augmented' is not defined

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).

In [59]:
import spacy
NLP_MODEL = spacy.load('en_core_web_sm')
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5, word_type=None):

    words = list(NLP_MODEL(sms.text))
    
    replacement_counter = 0
    for ind, word in enumerate(words):
        if replacement_counter >= num_replacements:
            break
        if word_type is None or word.pos_ != word_type:
            continue
        synonym = get_synonym(word.text)
        if synonym:
            words[ind] = synonym
            replacement_counter += 1
    else:
        print("Not enough replacements found")
        # I know it's not neccessary to display this, but I wanted to use the else statement after loop, and this is perfect place for it
    sms.text = ' '.join(word.text for word in words)
    return sms